In [1]:
import pandas as pd
import joblib

def predict_churn(new_data_dict):
    # Load trained model
    model = joblib.load('../models/random_forest_model.pkl')

    # Load original dataset for reference columns
    df = pd.read_csv('../data/WA_Fn-UseC_-Telco-Customer-Churn.csv')
    df = df.drop('customerID', axis=1)
    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
    df = df.dropna()

    # Encode binary columns
    binary_cols = ['gender', 'Partner', 'Dependents', 'PhoneService',
                   'PaperlessBilling', 'Churn']
    for col in binary_cols:
        df[col] = df[col].replace({'Yes': 1, 'No': 0, 'Male': 1, 'Female': 0})

    # One-hot encode categorical columns
    categorical_cols = ['MultipleLines', 'InternetService', 'OnlineSecurity',
                        'OnlineBackup', 'DeviceProtection', 'TechSupport',
                        'StreamingTV', 'StreamingMovies', 'Contract',
                        'PaymentMethod']
    df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

    # Process new customer data
    new_df = pd.DataFrame([new_data_dict])
    new_df = pd.get_dummies(new_df, columns=categorical_cols, drop_first=True)

    # Align new_df with training columns
    new_df = new_df.reindex(columns=df.drop('Churn', axis=1).columns, fill_value=0)

    # Predict
    pred = model.predict(new_df)[0]
    return "Churn Likely" if pred == 1 else "Not Likely to Churn"


In [2]:
new_customer = {
    'gender': 1,
    'SeniorCitizen': 0,
    'Partner': 1,
    'Dependents': 0,
    'tenure': 5,
    'PhoneService': 1,
    'PaperlessBilling': 1,
    'MonthlyCharges': 75.5,
    'TotalCharges': 350.0,
    'MultipleLines': 'No phone service',
    'InternetService': 'Fiber optic',
    'OnlineSecurity': 'No',
    'OnlineBackup': 'No',
    'DeviceProtection': 'Yes',
    'TechSupport': 'No',
    'StreamingTV': 'Yes',
    'StreamingMovies': 'Yes',
    'Contract': 'Month-to-month',
    'PaymentMethod': 'Electronic check'
}

predict_churn(new_customer)


C:\Users\user\AppData\Local\Temp\ipykernel_8620\2266541584.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace({'Yes': 1, 'No': 0, 'Male': 1, 'Female': 0})


'Not Likely to Churn'